This notebook explains how to use the diffthermo package to fit OCV models.

First, install the `diffthermo` package according to the README file. 

In [ ]:
conda create -n diffthermo python=3.11  torch=2.0.0 numpy=1.24 matplotlib pandas
conda activate diffthermo 
git clone https://github.com/BattModels/Diffthermo_OCV_paper.git
cd Diffthermo_OCV_paper
pip install .

To fit the OCV model, you need a csv file which contains the OCV VS SoC data. The first column of the file is SoC data, the second column is the OCV value. The OCV VS SoC data may come from experimental results or digitizing the results from figures of existing literature that contains OCV VS SoC information. 

In [1]:
import pandas as pd 
datafile_name = 'LFP.csv' # this is the csv file that you prepard, it should contain OCV VS SoC data
df = pd.read_csv(datafile_name,header=None)
print(df)

            0         1
0    0.993407  2.790879
1    0.988956  2.812509
2    0.986477  2.833543
3    0.985178  2.852396
4    0.983384  2.871134
..        ...       ...
169  0.062793  3.927804
170  0.062038  3.949528
171  0.061471  3.971788
172  0.060905  3.992601
173  0.059488  4.012022

[174 rows x 2 columns]


Now, you simply need to feed the prepared csv file that contains OCV VS SoC data into the diffthermo package. 

Put the name of your csv file as `datafile_name`. `number_of_Omegas` is a hyperparameter that you should choose. I suggest to start with small numbers (3 or 4 as examples), and increase the number if the fitted model does not look good. The `total_training_epochs` specifies how many round of updates the parameters (U0 and Omegas) will have. You should set the number accordingly, so that the loss value is minimized. I recommend the learning_rate to be 1000.0 (since the value of Omegas are usually large), you can leave this value unchanged. `loss_threshold` controls the minimum value of loss, below which the fitting process will automatically stop. 

A quick note: if your fitted results does not look good, TRY adjusting `G0_rand_range` and `Omegas_rand_range` in `train`. These two parameters control the initial guess of G0 and Omegas. Usually for an anode material, G0_rand_range=[-10*5000,-5*5000], 
 Omegas_rand_range=[-10*100,10*100] work well, and for cathode material, G0_rand_range=[-100*5000,-50*5000], Omegas_rand_range=[-100*100,100*100] work. 

Now let's just run the model and see what's going to happen:

In [2]:
from diffthermo.utils import train
params_list = train(datafile_name='LFP.csv', 
                        number_of_Omegas=4, 
                        learning_rate = 1000.0, 
                        total_training_epochs = 4000,
                        loss_threshold = 0.01,
                        G0_rand_range=[-100*5000,-50*5000], 
                        Omegas_rand_range=[-100*100,100*100])

Epoch   0  Loss 755.7955     Omega0 6756.0215 Omega1 -7991.9775 Omega2 7470.0220 Omega3 -9821.9785 G0 -392419.0000       
Epoch   1  Loss 733.1979     Omega0 5755.3140 Omega1 -8986.0186 Omega2 6478.4658 Omega3 -10815.3164 G0 -391419.3750       
Epoch   2  Loss 711.0432     Omega0 4756.7324 Omega1 -9964.8164 Omega2 5503.1030 Omega3 -11798.1689 G0 -390420.4062       
Epoch   3  Loss 689.4157     Omega0 3768.8916 Omega1 -10914.5898 Omega2 4552.1245 Omega3 -12767.6582 G0 -389422.3438       
Epoch   4  Loss 668.3527     Omega0 2798.9133 Omega1 -11816.6221 Omega2 3637.9133 Omega3 -13718.9951 G0 -388425.4688       
Epoch   5  Loss 647.8606     Omega0 1846.1133 Omega1 -12643.7529 Omega2 2781.3911 Omega3 -14643.9824 G0 -387430.0625       
Epoch   6  Loss 627.9399     Omega0 939.8514 Omega1 -13368.2920 Omega2 1990.6880 Omega3 -15542.5928 G0 -386436.4062       
Epoch   7  Loss 608.5592     Omega0 90.4020 Omega1 -13956.7627 Omega2 1285.6108 Omega3 -16409.0176 G0 -385444.7812       
Epoch   8  Loss

The loss value is decreasing in each epoch. Since we set the `total_training_epochs` to be 200, the fitting process stops at 200 epochs. You can adjust this number if you want to further minimize the loss value in order to get a more accurate OCV model.

Now, we have succesfully fitted an OCV model for graphite. To access your fitted model, call `write_ocv_functions` function:

In [3]:
from diffthermo.utils import write_ocv_functions
write_ocv_functions(params_list)

Found 1 phase coexistence region(s):
From x=0.0811861157417297 to x=0.9259547591209412





 Fitting Complete.

Fitted OCV function written in PyBaMM function (copy and paste readay!):

###################################

import numpy as np
import pybamm
from pybamm import exp, log, tanh, constants, Parameter, ParameterValues

def fitted_OCP(sto):
    _eps = 1e-7
    # rk params
    G0 = -336660.593750 # G0 is the pure substance gibbs free energy 
    Omega0 = 128051.375000 
    Omega1 = 61734.765625 
    Omega2 = -290618.718750 
    Omega3 = -163956.625000 
    Omegas =[Omega0, Omega1, Omega2, Omega3]
    # phase boundary 0
    x_alpha_0 = 0.0811861157417297
    x_beta_0 = 0.9259547591209412
    mu_coex_0 = -328734.6875000000000000
    is_outside_miscibility_gap_0 = (sto<x_alpha_0) + (sto>x_beta_0)
    # whether is outside all gap
    is_outside_miscibility_gaps = is_outside_miscibility_gap_0     
    mu_outside = G0 + 8.314*300.0*log((sto+_eps)/(1-sto+_eps))
    for i in range(0, le

You can directly copy & paste the fitted pybamm OCV function as printed in the terminal, or you can find the function in `fitted_ocv_functions.py`  file under the same directory where you put your csv file in. 